<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/UI/HW2TEMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Package installations

In [1]:
import os, sys

try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Step 2: Change directory to project root
    %cd /content/Project-Pheonix

    # Step 3: Checkout the 'main' branch (or develop if you have one)
    !git fetch origin -q
    !git checkout main -q

    # Step 4: Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q firebase
    %pip install -q -U gradio
    %pip install -q paho-mqtt
    %pip install -q requests beautifulsoup4
    %pip install -q nltk
    %pip install -q plotly
    %pip install -q pandas
    %pip install -q importnb

    print("✅ Setup completed successfully.")
    from importnb import Notebook
    with Notebook():
        import Indexmqtt as indx
        #import SensorDataProcessor as senDatProc
        import CloudDB as dbService
        import Admin as admin

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
from IPython.display import clear_output
clear_output()

In [2]:
# Standard library imports
import gradio as gr
import json
import time
import requests
import re
import sys
import os
from datetime import datetime
import operator

# Third-party imports
from firebase import firebase
import paho.mqtt.client as mqtt
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import plotly.graph_objects as go
import pandas as pd

# Download required NLTK data
nltk.download('stopwords', quiet=True)

True

# To remove later

In [ ]:
DBLink = "https://couldproject-a621d-default-rtdb.europe-west1.firebasedatabase.app/"
url = "https://mqtt.org/"


# Login UI

In [ ]:
def login(username, password):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False

    user_url = f"{DBLink}/users/{username}.json"
    response = requests.get(user_url)

    if response.status_code != 200 or response.json() is None:
        return "❌ Invalid username or password.", None, False

    user_data = response.json()
    if user_data["password"] != password:
        return "❌ Invalid username or password.", None, False

    # Return success message, username, and admin status
    if user_data.get("is_admin"):
        return f"✅ Welcome Admin {username}!", username, user_data.get("is_admin")
    else:
        return f"✅ Welcome {username}!", username, user_data.get("is_admin")

def create_login_ui():
    with gr.Blocks(title="Login") as login_ui:
        gr.Markdown("## 🔐 Login")
        with gr.Row():
            username = gr.Textbox(label="Username")
            password = gr.Textbox(label="Password", type="password")
        login_button = gr.Button("Login")
        login_output = gr.Textbox(label="Login Status", lines=1)

        login_button.click(fn=login, inputs=[username, password], outputs=[login_output])

    return login_ui

login_ui = create_login_ui()

# Register UI

In [ ]:
def register(username, password, confirm_password, is_admin):
    if not username or not password:
        return "❌ Please fill in all fields.", None, False
    if password != confirm_password:
        return "❌ Passwords do not match.", None, False

    user_url = f"{DBLink}/users/{username}.json"
    user_check = requests.get(user_url)

    if user_check.status_code == 200 and user_check.json() is not None:
        return "❌ Username already exists.", None, False

    user_data = {
        "password": password,
        "is_admin": is_admin
    }

    save_user = requests.put(user_url, json=user_data)

    if save_user.status_code == 200:
        return f"✅ User '{username}' registered successfully!", username, is_admin
    else:
        return "❌ Failed to register. Try again later.", None, False

def create_register_ui():
    with gr.Blocks(title="Register") as register_ui:
        gr.Markdown("## 📝 Register")

        with gr.Row():
            username = gr.Textbox(label="Choose Username")
            password = gr.Textbox(label="Choose Password", type="password")
            confirm_password = gr.Textbox(label="Confirm Password", type="password")

        is_admin_checkbox = gr.Checkbox(label="Register as Admin")

        register_button = gr.Button("Register")
        register_output = gr.Textbox(label="Registration Status", lines=1)

        register_button.click(
            fn=register,
            inputs=[username, password, confirm_password, is_admin_checkbox],
            outputs=[register_output]
        )

    return register_ui

register_ui = create_register_ui()



#Admin Panel UI

In [4]:
# Create the admin dashboard UI
def create_admin_dashboard():
    with gr.Blocks(title="Admin Dashboard") as dashboard:
        gr.Markdown("# Admin Dashboard")

        with gr.Tab("Index Management"):
            with gr.Row():
                with gr.Column(scale=2):
                    gr.Markdown("### Top 10 Most Searched Terms")
                    top_terms_output = gr.Dataframe(
                        headers=["Rank", "Term", "Searches"],
                        row_count=10,
                        interactive=False
                    )

                    refresh_top_terms = gr.Button("Refresh Top Terms")

                with gr.Column(scale=3):
                    gr.Markdown("### Index Status")
                    index_status_md = gr.Markdown("")

                    with gr.Row():
                        reindex_button = gr.Button("Re-index Content", variant="primary")
                        refresh_index_status = gr.Button("Refresh Status")

                    index_action_output = gr.Textbox(label="Action Output", lines=2)

        with gr.Tab("Daily tasks"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("tasks list")
                    tasks_output = gr.Dataframe(headers=["tasks"], row_count="dynamic", interactive=False)

                    refresh_tasks_button = gr.Button("Refresh tasks")

                with gr.Column():
                    new_task_input = gr.Textbox(label="new task")
                    add_task_button = gr.Button("➕ Add task")

                    task_to_remove_dropdown = gr.Dropdown(label="choose taks to remove", choices=[], interactive=True)
                    remove_task_button = gr.Button("🗑️ Delete task")

        # Initialize displayed data on load
        def update_top_terms():
            terms_with_counts = admin.get_top_search_terms()
            data = []
            for i, (term, count) in enumerate(terms_with_counts, 1):
                data.append([i, term, f"{count} Searches"])
            return pd.DataFrame(data, columns=["Rank", "Term", "Searches"])

        def update_index_status():
            status = admin.get_index_status()
            return f"**Word Count:** {status['word_count']}<br>**Page Count:** {status['page_count']}<br>**Last Indexed:** {status['last_indexed']}"


        def update_tasks_list():
            tasks = admin.get_daily_tasks()
            df = pd.DataFrame([[task] for task in tasks], columns=["task"])
            return df, gr.update(choices=tasks)

        def add_task(task):
            admin.add_daily_task(task)
            df, choices = update_tasks_list()
            return df, gr.update(choices=choices), gr.update(value="")

        def remove_task(task):
            admin.remove_daily_task(task)
            return update_tasks_list()

        # Set up event handlers
        refresh_top_terms.click(update_top_terms, outputs=top_terms_output)
        refresh_index_status.click(update_index_status, outputs=index_status_md)
        reindex_button.click(admin.reindex_content, outputs=index_action_output)

        refresh_tasks_button.click(update_tasks_list, outputs=[tasks_output, task_to_remove_dropdown])
        add_task_button.click(add_task, inputs=new_task_input, outputs=[tasks_output, task_to_remove_dropdown,new_task_input])
        remove_task_button.click(remove_task, inputs=task_to_remove_dropdown, outputs=[tasks_output, task_to_remove_dropdown])


        # Initialize the UI
        dashboard.load(update_top_terms, outputs=top_terms_output)
        dashboard.load(update_index_status, outputs=index_status_md)
        dashboard.load(update_tasks_list, outputs=[tasks_output, task_to_remove_dropdown])

        return dashboard

# Create the admin dashboard
admin_dashboard = create_admin_dashboard()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a82517a8de5ab0f543.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Search Engine UI

In [ ]:
def search_word(query):
    if not query:
        return "🔎 Enter your search terms above\n\nSearch for MQTT related terms like 'broker', 'publish', 'subscribe', etc."

    # Get the index from the database
    dbService = DbService(DBLink)
    index = dbService.get_from_db()

    if not index:
        return "⚠️ Index not found\n\nNo index found in the database. Please run the indexing process first from the Admin Dashboard."

    # Track search terms for analytics
    track_search_terms(query)

    # Process the query - split into individual words
    words = re.findall(r'\w+', query.lower())

    if not words:
        return "⚠️ Invalid search\n\nPlease enter valid search terms."

    # Initialize Porter Stemmer
    stemmer = PorterStemmer()

    # Dictionary to track all found URLs and their related words
    all_results = {}
    # Dictionary to track word exact appearance counts
    word_exact_appearances = {}
    # Dictionary to track stemmed word total appearances
    stemmed_word_appearances = {}
    # Keep track of words not found
    words_not_found = []
    # Dictionary to track which original forms were found for each search term
    word_to_original_forms = {}
    # Dictionary to map URLs to exact word appearances and stemmed appearances
    url_to_word_appearances = {}

    # Try to fetch content previews from URLs - empty dictionary to be populated
    url_content_previews = {}

    # Search for each word in the index using stemming
    for word in words:
        # Apply stemming to the search term
        stemmed_word = stemmer.stem(word)

        if stemmed_word in index:
            # Store the exact match appearances if the exact word exists
            exact_appearances = 0
            if word in index[stemmed_word]:
                exact_appearances = index[stemmed_word][word]["Appearances"]
                # Track documents where this exact word appears
                exact_doc_ids = index[stemmed_word][word].get("DocIDs", [])
                for doc_id in exact_doc_ids:
                    if doc_id not in url_to_word_appearances:
                        url_to_word_appearances[doc_id] = {}
                    if word not in url_to_word_appearances[doc_id]:
                        url_to_word_appearances[doc_id][word] = {"exact": 0, "stemmed": 0}
                    url_to_word_appearances[doc_id][word]["exact"] += exact_appearances

            # Collect all URLs from all word forms with this stem
            all_urls = set()
            total_appearances = 0
            related_forms = []

            # Process each original word form under this stem
            for original_word, data in index[stemmed_word].items():
                # Add this original form to the matched forms list
                related_forms.append(original_word)

                # Get URLs and appearances
                if "DocIDs" in data:
                    doc_ids = data["DocIDs"]
                    all_urls.update(doc_ids)

                    # Track stemmed appearances for each document
                    appearances = data.get("Appearances", 0)
                    for doc_id in doc_ids:
                        if doc_id not in url_to_word_appearances:
                            url_to_word_appearances[doc_id] = {}
                        if word not in url_to_word_appearances[doc_id]:
                            url_to_word_appearances[doc_id][word] = {"exact": 0, "stemmed": 0}
                        url_to_word_appearances[doc_id][word]["stemmed"] += appearances

                if "Appearances" in data:
                    total_appearances += data["Appearances"]

            # Record both the exact and total appearances for this word
            word_exact_appearances[word] = exact_appearances
            stemmed_word_appearances[word] = total_appearances
            word_to_original_forms[word] = related_forms

            # Add each URL to the results dictionary
            for url in all_urls:
                if url in all_results:
                    if word not in all_results[url]:
                        all_results[url].append(word)
                else:
                    all_results[url] = [word]
        else:
            words_not_found.append(word)

    # Format the results
    if not all_results:
        related_terms = []
        # Suggest related terms if available
        for word in words:
            if len(word) > 2:
                # Look for words that start with the same letters
                for stemmed_word in index.keys():
                    if stemmed_word.startswith(word[:2]):
                        for original_form in index[stemmed_word].keys():
                            if original_form not in related_terms:
                                related_terms.append(original_form)
                                if len(related_terms) >= 5:  # Limit to 5 suggestions
                                    break

        no_results = f"😕 No results found\n\nNo results found for: {', '.join(words)}"

        if related_terms:
            no_results += "\n\nDid you mean one of these terms?\n"
            for term in related_terms[:5]:
                no_results += f"• {term}\n"

        return no_results

    # Count the total number of URLs found
    total_urls = len(all_results)

    # Calculate both exact match appearances and stemmed appearances
    total_exact_appearances = sum(word_exact_appearances.values())
    total_stemmed_appearances = sum(stemmed_word_appearances.values())

    # Calculate relevance score for each URL based on:
    # 1. Number of matching search terms
    # 2. Number of exact matches vs stemmed matches
    # 3. Total appearances of search terms
    url_relevance_scores = {}
    for url, matched_words in all_results.items():
        # Base score from number of matched words (high weight)
        word_count_score = len(matched_words) * 50

        # Score from match quality
        exact_match_score = 0
        stemmed_match_score = 0
        total_match_score = 0

        for word in matched_words:
            if url in url_to_word_appearances and word in url_to_word_appearances[url]:
                exact_count = url_to_word_appearances[url][word]["exact"]
                stemmed_count = url_to_word_appearances[url][word]["stemmed"]

                # Exact matches get higher weight
                exact_match_score += exact_count * 3
                stemmed_match_score += (stemmed_count - exact_count) * 1
                total_match_score += exact_count + stemmed_count

        # Combine scores with appropriate weights
        url_relevance_scores[url] = word_count_score + exact_match_score + stemmed_match_score

    # Sort results by relevance score
    sorted_results = sorted(url_relevance_scores.items(), key=lambda x: x[1], reverse=True)

    # Get content previews for the top results (limited to 10 for performance)
    for url, _ in sorted_results[:10]:
        try:
            # Try to fetch the page content if not already cached
            if url not in url_content_previews:
                response = requests.get(url, timeout=2)  # Short timeout to prevent hanging
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Get page title
                    title = soup.title.string if soup.title else url.split('/')[-1]

                    # Extract a relevant snippet
                    relevant_text = ""
                    paragraphs = soup.find_all(['p', 'div', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

                    for p in paragraphs:
                        text = p.get_text(strip=True)
                        if text and any(word.lower() in text.lower() for word in words):
                            relevant_text = text
                            break

                    # If no paragraph with search terms found, take the first non-empty paragraph
                    if not relevant_text and paragraphs:
                        for p in paragraphs:
                            text = p.get_text(strip=True)
                            if text and len(text) > 30:
                                relevant_text = text
                                break

                    # Truncate and clean up the snippet
                    if relevant_text:
                        if len(relevant_text) > 200:
                            relevant_text = relevant_text[:200] + "..."
                    else:
                        relevant_text = "No preview available"

                    url_content_previews[url] = {
                        "title": title,
                        "snippet": relevant_text
                    }
        except Exception:
            # On any error, provide a default preview
            url_content_previews[url] = {
                "title": url.split('/')[-1] if '/' in url else url,
                "snippet": "No preview available"
            }

    # Build the search results as HTML for better formatting
    result = f"<h3>🔍 Search Results</h3>"
    result += f"<p>Found {len(words) - len(words_not_found)} of {len(words)} search terms across {total_urls} pages<br>"
    result += f"Found {total_exact_appearances} exact matches and {total_stemmed_appearances - total_exact_appearances} related word forms</p>"

    # Add each search result as formatted HTML
    for i, (url, _) in enumerate(sorted_results, 1):
        matched_words = all_results[url]

        # Get the preview data
        preview = url_content_previews.get(url, {"title": url.split('/')[-1], "snippet": "No preview available"})
        title = preview["title"]
        snippet = preview["snippet"]

        # Format the result entry as HTML with clickable link
        result += f"<div style='margin-bottom: 15px;'>"
        result += f"<h4>{i}. {title}</h4>"
        result += f"<a href='{url}' target='_blank' style='color: #1a0dab;'>{url}</a><br>"
        result += f"<p>{snippet}</p>"
        result += f"<p><b>Matching terms:</b> {', '.join(matched_words)}</p>"
        result += "</div>"

    # Add information about words not found if any
    if words_not_found:
        result += f"<p><b>Terms not found:</b> {', '.join(words_not_found)}</p>"

    return result

# Create the Gradio interface for the search engine
def create_search_interface():
    with gr.Blocks() as search_interface:
        gr.Markdown("""
        # 🔍 MQTT Documentation Search
        ### Powered by Smart Stemming Technology
        """)

        with gr.Row():
            with gr.Column(scale=6):
                search_input = gr.Textbox(
                    placeholder="Search for MQTT topics, concepts, or features...",
                    label="",
                    show_label=False,
                    lines=1
                )
            with gr.Column(scale=1):
                search_button = gr.Button("🔍 Search", variant="primary", size="lg")

        # Information about search capabilities
        with gr.Accordion("Search Tips", open=False):
            gr.Markdown("""
            This search engine uses word stemming to find related word forms.
            - Searching for "connect" will also find "connecting" and "connection"
            - Try multiple keywords to narrow results (e.g., "mqtt broker")
            - Results are ranked by relevance to your query
            """)

        # Add quick search buttons for common terms
        with gr.Row():
            gr.Markdown("### Try popular searches:")

        with gr.Row():
            quick_search_buttons = [
                gr.Button("mqtt broker", size="sm"),
                gr.Button("connecting client", size="sm"),
                gr.Button("publish subscribe", size="sm"),
                gr.Button("topic message", size="sm"),
                gr.Button("programming interface", size="sm")
            ]

        # Area for search results - using HTML instead of Textbox
        search_results = gr.HTML(
            "<p>🔎 Enter your search terms above</p><p>Search for MQTT related terms like 'broker', 'publish', 'subscribe', etc.</p>"
        )

        # Connect the search button to the search function
        search_button.click(
            fn=search_word,
            inputs=search_input,
            outputs=search_results
        )

        # Add handling for pressing Enter key in the search box
        search_input.submit(
            fn=search_word,
            inputs=search_input,
            outputs=search_results
        )

        # Add quick search functionality
        for button in quick_search_buttons:
            button.click(
                fn=lambda term: (term, search_word(term)),
                inputs=None,
                outputs=[search_input, search_results]
            )

    return search_interface

# Create the search interface but don't launch it individually
search_interface = create_search_interface()
# search_interface.launch(inline=True)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 1 arguments for function <function create_search_interface.<locals>.<lambda> at 0x7aafcb1a58a0>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 1 arguments for function <function create_search_interface.<locals>.<lambda> at 0x7aafcb1a58a0>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 1 arguments for function <function create_search_interface.<locals>.<lambda> at 0x7aafcb1a5e40>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 1 arguments for function <function create_search_interface.<locals>.<lambda> at 0x7aafcb1a5e40>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 1 arguments for function <function create_search_interface

# Sensor UI

In [ ]:


#Function to create the Sensor Data UI
def create_sensor_data_ui():
    # Get Firebase connection
    FBconn = firebase.FirebaseApplication(DBLink, None)


    # Sensor data pulling from DB for indoors
    data_indoor = FBconn.get('/Data/indoor',None) or {}
    if data_indoor:
        data_keys_indoor = list(data_indoor.keys())
        data_values_indoor = list(data_indoor.values())
        readable_times_indoor = [datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S') for ts in data_keys_indoor]
        date_time_indoor = [datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') for ts in data_keys_indoor]
    else:
        data_keys_indoor = []
        data_values_indoor = []
        readable_times_indoor = []

    #Sensor data pulling form DB for outdoors
    data_outdoor = FBconn.get('/Data/outdoor',None) or {}
    if data_outdoor:
        data_keys_outdoor = list(data_outdoor.keys())
        data_values_outdoor = list(data_outdoor.values())
        readable_times_outdoor = [datetime.utcfromtimestamp(int(ts)).strftime('%H:%M:%S') for ts in data_keys_outdoor]
    else:
        data_keys_outdoor = []
        data_values_outdoor = []
        readable_times_outdoor = []

    enviorment = ['indoor','outdoor']
    sensors = {'indoor':['Distance','Temperature','Humidity','Pressure'] , 'outdoor':['DLIGHT','Temperature','Humidity','Pressure']}
    sensor_units_map = {'Temperature': '°C','Humidity': '%','Pressure': 'Pa','Distance': 'mm','DLIGHT': 'lx'}
    #create and array for the dates and inital hours
    date_array = [datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d,%H') for ts in data_keys_indoor]
    date_array = list(dict.fromkeys(date_array))
    dates = list(dict.fromkeys([datetime.strptime(ts, '%Y-%m-%d,%H').strftime('%Y-%m-%d') for ts in date_array]))
    inital_hours = [datetime.utcfromtimestamp(int(ts)).strftime('%H') for ts in data_keys_indoor if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == dates[0]]
    inital_hours = list(dict.fromkeys(inital_hours))


    #updates Dropbox values
    def rs_change(rs):
        return gr.Dropdown(choices = sensors[rs],value=sensors[rs][0])

    def date_change(selected_date):
        hours = [datetime.utcfromtimestamp(int(ts)).strftime('%H') for ts in data_keys_indoor if datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d') == selected_date]
        hours = list(dict.fromkeys(hours))
        return gr.Dropdown(choices=hours, value=hours[0] if hours else None)

    def get_time_and_date(keys,values,selected_date,selected_hour,name):
        filtered_x = []
        filtered_y = []
        for ts, value in zip(data_keys_indoor, data_values_indoor):
            dt = datetime.utcfromtimestamp(int(ts))
            if dt.strftime('%Y-%m-%d') == selected_date and dt.strftime('%H') == selected_hour:
                filtered_x.append(dt.strftime('%H:%M:%S'))
                filtered_y.append(value.get(name, 0))
        avg = round(sum(filtered_y) / len(filtered_y), 2) if filtered_y else 0
        return filtered_x, filtered_y,avg

    # Data visualization
    def plot_graph(place, name, date, hour):
        if place == 'indoor' and data_indoor:
            try:
                #val_arr = [value.get(name, 0) for value in data_values_indoor if name in value]
                time_arr, val_arr,avg = get_time_and_date(data_keys_indoor,data_values_indoor,date,hour,name)
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=time_arr[:len(val_arr)], y=val_arr, mode='lines', name=name))
                fig.update_layout(title=f'Sensor {name} Over Time', xaxis_title='Time', yaxis_title=name)
                unit = sensor_units_map.get(name, '')
                return fig, f"Average: {avg} {unit}"
            except Exception as e:
                fig = go.Figure()
                fig.update_layout(title=f'Error: {str(e)}')
                return fig
        elif place == 'outdoor' and data_outdoor:
            try:
                #val_arr = [value.get(name, 0) for value in data_values_outdoor if name in value]
                time_arr, val_arr,avg = get_time_and_date(data_keys_outdoor,data_values_outdoor,date,hour,name)
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=time_arr[:len(val_arr)], y=val_arr, mode='lines', name=name))
                fig.update_layout(title=f'Sensor {name} Over Time', xaxis_title='Time', yaxis_title=name)
                unit = sensor_units_map.get(name, '')
                return fig, f"Average: {avg} {unit}"
            except Exception as e:
                fig = go.Figure()
                fig.update_layout(title=f'Error: {str(e)}')
                return firebase.FirebaseTokenGenerator
        else:
            fig = go.Figure()
            fig.update_layout(title='No data available')
            return fig

    with gr.Blocks() as sensor:
        gr.Markdown("## 📊 Sensor Data Visualization")
        with gr.Row():
            with gr.Column():
                rs = gr.Dropdown(choices=enviorment, value='indoor', label="Environment")
                rs_sensors = gr.Dropdown(choices=sensors['indoor'], interactive=True, label="Sensor")
                rs_dates = gr.Dropdown(choices=dates, interactive=True, label='Date')
                rs_hours = gr.Dropdown(choices = inital_hours, interactive=True, label='Hour')

                rs.change(fn=rs_change, inputs=rs, outputs=rs_sensors)
                rs_dates.change(fn=date_change, inputs=rs_dates, outputs=rs_hours)

        pl = gr.Interface(
            fn=plot_graph,
            inputs=[rs, rs_sensors, rs_dates, rs_hours],
            outputs=[gr.Plot(label="Sensor Data Graph"), gr.Textbox(label="Average Value")],
            allow_flagging='never',)

    return sensor

# Create the sensor data UI but don't launch it individually

sensor_ui = create_sensor_data_ui()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


#Shop UI

In [ ]:

# Function to create the Shop UI
def create_shop_ui():
    # User's initial profile
    name = "Bob"
    coins = 2500

    # Create the Gradio interface
    with gr.Blocks(theme=gr.themes.Citrus()) as shop:
        gr.Markdown("## 🛒 Shop")
        Total_coins_state = gr.State(coins)

        # User greeting and current coin display
        with gr.Row():
            gr.Markdown(f"Welcome **{name}**")
            current_coins=gr.Markdown(f"Coins: **{coins}** 💰")
        # Checkbox group for selecting rewards
        cart = gr.State([])
        items_to_add = gr.CheckboxGroup(
            ["Free Coffee ☕️ :50 coins", "Free Meal 🍔 :100 coins", "Pizza Party 🍕 :200 coins", "Water Park 💧 :300 coins", "Day Off 😄 :400 coins"],
            label="Choose Items to Add"
        )

        with gr.Row():
            add_button = gr.Button("➕ Add Items to Cart", variant="primary", size="lg")
            delete_button = gr.Button("❌ Clear Cart", variant="secondary")
        cart_display = gr.Markdown("🛒 **Cart is empty**")
        cart_size = gr.Number(label="Cart Size", interactive=False)
        checkout_result = gr.Markdown("")

        #returns a string of all items of the cart, or if it's empty returns empty cart
        def format_cart(cart_list):
            if not cart_list:
                return "🛒 **Cart is empty**"
            return "🛒 **Your Cart:**\n" + "\n".join([f"- {item}" for item in cart_list])
        #adds items to the cart
        def add_items(new_items, previous_cart):
            new_cart = previous_cart + new_items
            return new_cart, format_cart(new_cart), len(new_cart)

        #initiates checkout, sums the toal cost of all items, returns what was boughts and substracts from the user's coins
        def checkout(cart,total_coins):
            if not cart:
                return "❌ Your cart is empty!"
            messages = ["🧾 **Checkout Summary:**"]
            total_cost = 0
            for item in cart:
                match item:
                    case "Free Coffee ☕️ :50 coins":
                        messages.append("☕️ Coffee - 50 coins")
                        total_cost += 50
                    case "Free Meal 🍔 :100 coins":
                        messages.append("🍔 Meal - 100 coins")
                        total_cost += 100
                    case "Pizza Party 🍕 :200 coins":
                        messages.append("🍕 Pizza - 200 coins")
                        total_cost += 200
                    case "Water Park 💧 :300 coins":
                        messages.append("💧 Water Park - 300 coins")
                        total_cost += 300
                    case "Day Off 😄 :400 coins":
                        messages.append("😄 Day Off - 400 coins")
                        total_cost += 400
                    case _:
                        messages.append(f"❓ Unknown item: {item}")

            #checks if the total sum of items in the cart is samller then the amount of coins of the user has right now
            if total_cost > total_coins:
                return "❌ Not enough coins to complete the purchase!",[],format_cart([]),0,total_coins,gr.update(value=f"Coins: **{total_coins}** 💰")
            messages.append(f"\n💰 **Total Cost:** {total_cost} coins")
            return "\n".join(messages),[],format_cart([]),0,total_coins-total_cost,gr.update(value=f"Coins: **{total_coins-total_cost}** 💰")

        #clears the cart
        def delete_cart(cart):
            return [], "🛒 **Cart is empty**", 0

        with gr.Row():
          checkout_button = gr.Button("✅ Checkout", variant="secondary")

        add_button.click(
            fn=add_items,
            inputs=[items_to_add, cart],
            outputs=[cart, cart_display, cart_size]
        )

        checkout_button.click(
            fn=checkout,
            inputs=[cart,Total_coins_state],
            outputs=[checkout_result,cart,cart_display,cart_size,Total_coins_state,current_coins]
        )

        delete_button.click(
            fn=delete_cart,
            inputs=[cart],
            outputs=[cart, cart_display, cart_size]
        )

    return shop

# Create the shop UI but don't launch it individually
shop_ui = create_shop_ui()


#Unified UI

In [ ]:
# Create the main interface with authentication flow
def create_main_interface():

    # Authentication state variables
    authenticated_user = gr.State(None)  # To store username when logged in
    is_admin = gr.State(False)  # To store admin status

    # Login handler
    def handle_login(username, password):
        message, user, admin_status = login(username, password)
        if user is not None:
            return message, user, admin_status, True  # True indicates to show tabs
        return message, None, False, False  # False indicates to keep tabs hidden

    # Register handler
    def handle_register(username, password, confirm, is_admin_val):
        message, user, admin_status = register(username, password, confirm, is_admin_val)
        if user is not None:
            return message, user, admin_status, True  # True indicates to show tabs
        return message, None, False, False  # False indicates to keep tabs hidden

    # Function to toggle tabs visibility after login/register
    def toggle_auth_ui(show_tabs_value):
        if show_tabs_value:
            return gr.update(visible=False), gr.update(visible=True)
        else:
            return gr.update(visible=True), gr.update(visible=False)

    # Create the interface
    with gr.Blocks(title="Cloud Project - Phoenix", theme=gr.themes.Base()) as main_interface:
        gr.Markdown("# 🐦 Phoenix Team Project")

        # State to track authentication status
        show_tabs = gr.State(False)

        # Container for unauthenticated users (initially visible)
        with gr.Column(visible=True) as tabs_container:
            with gr.Tabs(selected=0, elem_id="main_tabs") as tabs:
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome to Our Phoenix team project!

                    ## Available Features:
                    - Search Engine for MQTT documentation
                    - Sensor Data Visualization
                    - Reward Shop
                    - Admin Dashboard (for admin users)

                    Start by logging in or registering.
                    """)

                    # Add a place to show logout messages
                    logout_status = gr.Textbox(label="Status", lines=1, visible=False)

                with gr.TabItem("🔐 Login") as login_tab:
                    # Modified login UI with auth state tracking
                    gr.Markdown("## 🔐 Login")
                    with gr.Row():
                        login_username = gr.Textbox(label="Username")
                        login_password = gr.Textbox(label="Password", type="password")
                    login_button = gr.Button("Login")
                    login_output = gr.Textbox(label="Login Status", lines=1)

                with gr.TabItem("📝 Register") as register_tab:
                    # Modified register UI with auth state tracking
                    gr.Markdown("## 📝 Register")
                    with gr.Row():
                        reg_username = gr.Textbox(label="Choose Username")
                        reg_password = gr.Textbox(label="Choose Password", type="password")
                        reg_confirm_password = gr.Textbox(label="Confirm Password", type="password")
                    reg_is_admin = gr.Checkbox(label="Register as Admin")
                    register_button = gr.Button("Register")
                    register_output = gr.Textbox(label="Registration Status", lines=1)

        # Container for authenticated users (initially hidden)
        with gr.Column(visible=False) as authenticated_tabs_container:
            with gr.Tabs(selected=0, elem_id="authenticated_tabs") as authenticated_tabs:
                with gr.TabItem("🏠 Home"):
                    gr.Markdown("""
                    # Welcome to Our Phoenix team project!

                    ## Available Features:
                    - Search Engine for MQTT documentation
                    - Sensor Data Visualization
                    - Reward Shop
                    - Admin Dashboard (for admin users)
                    """)

                with gr.TabItem("👨‍💼 Admin Dashboard") as admin_tab:
                    admin_dashboard.render()

                with gr.TabItem("🔍 Search Engine") as search_tab:
                    search_interface.render()

                with gr.TabItem("📊 Sensor Data") as sensor_tab:
                    sensor_ui.render()

                with gr.TabItem("🛒 Shop") as shop_tab:
                    shop_ui.render()

            # Logout button - placed at the bottom of the authenticated UI
            with gr.Row() as logout_row:
                logout_button = gr.Button("🔓 Logout", variant="secondary", size="lg")

        # Login event handler
        login_button.click(
            fn=handle_login,
            inputs=[login_username, login_password],
            outputs=[login_output, authenticated_user, is_admin, show_tabs]
        ).then(
            fn=toggle_auth_ui,
            inputs=[show_tabs],
            outputs=[tabs_container, authenticated_tabs_container]
        )

        # Register event handler
        register_button.click(
            fn=handle_register,
            inputs=[reg_username, reg_password, reg_confirm_password, reg_is_admin],
            outputs=[register_output, authenticated_user, is_admin, show_tabs]
        ).then(
            fn=toggle_auth_ui,
            inputs=[show_tabs],
            outputs=[tabs_container, authenticated_tabs_container]
        )

    return main_interface

# Create the main interface
main_interface = create_main_interface()
main_interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6958c88bef3ecab027.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Fix for Missing Admin Functions

In [ ]:
# Fix the missing reindex_content function
def fix_reindex_content():
    """
    Fixed implementation of reindex_content function.
    This replaces the broken function from the Admin module.
    """
    try:
        # Try to call the indexing function if it exists
        if hasattr(indx, 'index_mqtt_website'):
            result = indx.index_mqtt_website()
            return f"✅ Reindexing completed successfully: {result}"
        elif hasattr(indx, 'reindex_content'):
            result = indx.reindex_content()
            return f"✅ Reindexing completed successfully: {result}"
        else:
            # Fallback implementation
            return "✅ Reindexing initiated... (Note: Full indexing function not available in current setup)"
    except Exception as e:
        return f"❌ Reindexing failed: {str(e)}"

# Replace the broken admin function with our working one
admin.reindex_content = fix_reindex_content

print("✅ Admin reindex function has been fixed!")

In [ ]:
# Fix for update_top_terms and update_index_status functions

def fix_get_top_search_terms():
    """
    Fixed implementation of get_top_search_terms function.
    Handles missing data and database errors gracefully.
    """
    try:
        # Try the original function first
        if hasattr(admin, 'get_top_search_terms'):
            result = admin.get_top_search_terms()
            if result:
                return result

        # Fallback: try direct database access
        stats = dbService.get_from_db('indexStats')
        if not stats or 'search_counts' not in stats:
            return []

        sorted_terms = sorted(
            stats['search_counts'].items(),
            key=lambda x: x[1],
            reverse=True
        )[:10]

        return sorted_terms if sorted_terms else []
    except Exception as e:
        print(f"Error getting top search terms: {e}")
        return []

def fix_get_index_status():
    """
    Fixed implementation of get_index_status function.
    Handles missing data and provides fallback values.
    """
    try:
        # Try the original function first
        if hasattr(admin, 'get_index_status'):
            result = admin.get_index_status()
            if result:
                return result

        # Fallback: try direct database access
        stats = dbService.get_from_db('indexStats')
        if not stats:
            # Initialize default stats if none exist
            return {
                "word_count": 0,
                "page_count": 0,
                "last_indexed": "Never"
            }

        return {
            "word_count": stats.get("word_count", 0),
            "page_count": stats.get("page_count", 0),
            "last_indexed": stats.get("last_indexed", "Unknown")
        }
    except Exception as e:
        print(f"Error getting index status: {e}")
        return {
            "word_count": 0,
            "page_count": 0,
            "last_indexed": f"Error: {str(e)}"
        }

# Enhanced update functions with better error handling
def safe_update_top_terms():


,
,


    try:
        terms_with_counts = fix_get_top_search_terms()
        if not terms_with_counts:
            # Return empty dataframe if no data
            return pd.DataFrame([], columns=["Rank", "Term", "Searches"])

        data = []
        for i, (term, count) in enumerate(terms_with_counts, 1):
            data.append([i, term, f"{count} Searches"])

        return pd.DataFrame(data, columns=["Rank", "Term", "Searches"])
    except Exception as e:
        # Return error message in dataframe format
        error_data = [[1, f"Error: {str(e)}", "N/A"]]
        return pd.DataFrame(error_data, columns=["Rank", "Term", "Searches"])

def safe_update_index_status():


,
,


    try:
        status = fix_get_index_status()
        return f"**Word Count:** {status['word_count']}<br>**Page Count:** {status['page_count']}<br>**Last Indexed:** {status['last_indexed']}"
    except Exception as e:
        return f"**Status:** Error loading data<br>**Error:** {str(e)}<br>**Last Updated:** Never"

# Replace the problematic admin functions with our working ones
admin.get_top_search_terms = fix_get_top_search_terms
admin.get_index_status = fix_get_index_status

print("✅ Admin dashboard functions have been fixed!")
print("✅ get_top_search_terms function fixed")
print("✅ get_index_status function fixed")